# Deep Learning con Python

Ejercicio final para la asignatura de Deep Learning en el Máster de Big Data & Data Science. Esta prueba consiste en la predicción de una variable objetivo, que será el considerar si el nivel de **ozono** está elevado o no. Se trata de un problema de clasificación binaria. 

Para ello, generaremos un modelo de **Red Neuronal Recurrente** que prediga esta variable.

## Recurrent Neural Networks
En las Redes Neuronales Recurrentes guardamos la salida de las activaciones de una o más capas de nuestra red. Habitualmente estas son activaciones ocultas. Entonces, la próxima vez que devolvemos el input a la red, incluimos el output anteriormente guardado dentro de los outputs normales de la capa anterior. Por ejemplo, si una capa oculta tiene 10 nodos de input regulares y 128 nodos coultos en la capa, entonces tendrá 138 nodos totales. Si es la primera vez que se intenta alimentar esa capa, entonces habrá que rellenar esos 128 inputos con 0s.

![Figura 1](https://cdn-images-1.medium.com/max/1600/1*NKhwsOYNUT5xU7Pyf6Znhg.png)

## El problema de Vanishing gradient

Este problema es la dificultad encontrada en el entrenamiento de redes neuronales artificales basadas en aprendizaje gradiente y backpropagation. En estos métodos, cada peso de la red neuronal recibe una actualización proporcional al la derivada de la función de error respecto al peso actual en cada iteración del entrenamiento. El problema en algunos casos es, el gradiente será muy pequeño, evitando que el peso cambie de valor. En el peor de los casos, esto puede detener completamente la red neuronal de un entrenamiento posterior. Como un ejemplo de la causa del problema, las funciones tales como la función tangente hiperbólica, tienen gradientes en el range (0,1), y la propagación hacia atrás calcula los gradientes por la regla de la cadena. Esto tiene un efecto de multipllicar ``n`` de estos pequeños números para calcular gradientes de las capas "frontales" en una red de ``n`` capas, lo que significa que el gradiente (o la señal de erorr) disminuye exponencialmente con ``n`` mientras que las capas frontales se entrenan muy lentamente

![Figura 2](https://cdn-images-1.medium.com/max/1460/1*FWy4STsp8k0M5Yd8LifG_Q.png)

## Long Short Term Memory (LSTM)

Los bloques (o unidades) de LSTM son una construicción de capas de una red recurrente neuronal (RNN). Una RNN compuesta de unidades LSTM se le llama una LSTM network. Una unidad común de LSTM está compuesta por una célula, una puerta de input, una puerta de output y una puerta de olvidado (forget gate). La célula es responsable de "recordar" los valores de los valores del intervalo arbitrarios, de ahí la "memoria" de LSTM. Cada una de las 3 puertas anteriormente nombradas, son lo que conocemos como una "neurona artificial convencional", como una red neuronal multicapa. Lo que hace es, computar una activación (usando la función de activación) de una suma de pesos. Intuitivamente, pueden nombrarse como los reguladores del flujo de los valores (flow of values) que va a través de las LSTM, de ahí la denominación de "puerta". Hay conexiones entre esta puerta y la célula.

La expresión largo-corto-plazo (long short term) se refiere al hecho de que las redes LSTM son un modelo de memoria a corto y largo plazo las cuales duran un periodo de tiempo. Una LSTM está bien optimizada para clasificar, procesar y predecir series temporales, dados unos periodos de tiempo de un tamaño desconocido y una duración entre los eventos importantes. Las LSTM han sido desarrolladas para tratar con el problema del``exploding`` y ``vanishing`` ``gradient``cuando se entrenan las redes neuronales tradicionales.

![Figura 3](https://cdn-images-1.medium.com/max/1600/0*LyfY3Mow9eCYlj7o.)

### Componentes de una LSTM
 * Forget gate "f" (una red neuronal con sigmoide)
 * Candidate layer "C" (una RN con Tanh)
 * Input Gate "I" (Una RN con sigmoide)
 * Ouput Gate "O" (Una RN con sigmoide)
 * Hidden State "H" (Un vector)
 * Memory State "C" (Un vector)
 * Inputs de la célula LSTMen cualquier estado son X<sub>t</sub> (input actual) , H<sub>t-1</sub> (estado anterior oculto) y C<sub>t-1</sub> (estado de la memoria previo.
 * Outputs de la célula LSTM son H<sub>t</sub> (estado oculto actual) y C<sub>t</sub> (estado de la memoria actual).



### Cargamos las librerías de trabajo

## Carga de Librerías de trabajo

In [2]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print(os.listdir("/home/jmgonzalezro/jmgonzalezro@gmail.com/Data Analysis/Universidad/17 Deep Learning/Practica"))

['onehr.data', 'ozone.csv', 'eighthr.names', 'onehr.names', 'Practica.ipynb', 'eighthr.data']


## Carga del dataset

In [7]:
ozoneDF = pd.read_csv("/home/jmgonzalezro/jmgonzalezro@gmail.com/Data Analysis/Universidad/17 Deep Learning/Practica/ozone.csv", sep = ",")

## Preprocesado y creación de datasets

In [10]:
ozoneDF.head()

,Date,WSR0,WSR1,WSR2,WSR3,WSR4,WSR5,WSR6,WSR7,WSR8,...,RH50,U50,V50,HT50,KI,TT,SLP,SLP_,Precp,target
0,1/1/1998,0.8,1.8,2.4,2.1,2,2.1,1.5,1.7,1.9,...,0.15,10.67,-1.56,5795,-12.1,17.9,10330,-55,0,0.0
1,1/2/1998,2.8,3.2,3.3,2.7,3.3,3.2,2.9,2.8,3.1,...,0.48,8.39,3.84,5805,14.05,29,10275,-55,0,0.0
2,1/3/1998,2.9,2.8,2.6,2.1,2.2,2.5,2.5,2.7,2.2,...,0.6,6.94,9.8,5790,17.9,41.3,10235,-40,0,0.0
3,1/4/1998,4.7,3.8,3.7,3.8,2.9,3.1,2.8,2.5,2.4,...,0.49,8.73,10.54,5775,31.15,51.7,10195,-40,2.08,0.0
4,1/5/1998,2.6,2.1,1.6,1.4,0.9,1.5,1.2,1.4,1.3,...,?,?,?,?,?,?,?,?,0.58,0.0


Obseravmos que tenemos las siguientes variables importantes:

O 3 - Local ozone peak prediction

Upwind - Upwind ozone background level

EmFactor - Precursor emissions related factor

Tmax - Maximum temperature in degrees F

Tb - Base temperature where net ozone production begins (50 F)

SRd - Solar radiation total for the day

WSa - Wind speed near sunrise (using 09-12 UTC forecast mode)

WSp - Wind speed mid-day (using 15-21 UTC forecast mode)



Además de:

KI - K-Index

TT - T-Totals

SLP - Sea level pressure

SLP_ - SLP change from previous day

Además de la target, que sería nuestro último atributo llamado **"target"**

In [19]:
# Compruebo que no tiene duplicados
print("El número de duplicados es:\n",ozoneDF.duplicated().sum(),
      "\n\nLos tipos de varibales son:\n", ozoneDF.dtypes)

# Compruebo los valores de cada una de las variables
# ozoneDF.dtypes,

# Evidentemente son todas objetos excepto la variable objetivo que es un numero

El número de duplicados es:
 0 

Los tipos de varibales son:
 Date       object
WSR0       object
WSR1       object
WSR2       object
WSR3       object
           ...   
TT         object
SLP        object
SLP_       object
Precp      object
target    float64
Length: 74, dtype: object


In [15]:
# Compruebo nulls
ozoneDF.isnull().sum()

Date      0
WSR0      0
WSR1      0
WSR2      0
WSR3      0
         ..
TT        0
SLP       0
SLP_      0
Precp     0
target    0
Length: 74, dtype: int64

In [20]:
ozoneDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2534 entries, 0 to 2533
Data columns (total 74 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    2534 non-null   object 
 1   WSR0    2534 non-null   object 
 2   WSR1    2534 non-null   object 
 3   WSR2    2534 non-null   object 
 4   WSR3    2534 non-null   object 
 5   WSR4    2534 non-null   object 
 6   WSR5    2534 non-null   object 
 7   WSR6    2534 non-null   object 
 8   WSR7    2534 non-null   object 
 9   WSR8    2534 non-null   object 
 10  WSR9    2534 non-null   object 
 11  WSR10   2534 non-null   object 
 12  WSR11   2534 non-null   object 
 13  WSR12   2534 non-null   object 
 14  WSR13   2534 non-null   object 
 15  WSR14   2534 non-null   object 
 16  WSR15   2534 non-null   object 
 17  WSR16   2534 non-null   object 
 18  WSR17   2534 non-null   object 
 19  WSR18   2534 non-null   object 
 20  WSR19   2534 non-null   object 
 21  WSR20   2534 non-null   object 
 22  

In [21]:
ozoneDF.describe()

# Con este describe compruebo que no hay valores que estén fuera de rango 0-1 en la variable target.

,target
count,2534.000000
mean,0.063141
std,0.243265
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000
